## Gradient Descent (Linear Regression and Logistic Regression)

In [4]:
import numpy as np
np.random.seed(0)


## Linear Regression using Gradient Descent

In [2]:
def genData(size, noise_mean = None, noise_sd = None):
    x1 = np.random.uniform(10, 20, size)
    x2 = np.random.uniform(1,5, size)

    # True formula: y = 2 + x1*5 + x2*4 + noise
    true_weights = [2, 5, 4]

    # Add the "x_0 = 1" (called the intercept term) so that we can use matrix multiplication
    X = np.vstack((np.ones(size), x1, x2))

    y = np.dot(true_weights, X)

    if noise_mean is not None and noise_sd is not None:
        noise = np.random.normal(noise_mean, noise_sd, size)
        y = y + noise
    
    return X, y

### Some background stuff

In [41]:
size = 10
X, y = genData(size)
print "True formula: y = 2 + x1*5 + x2*4 + noise (if needed)"
print
print "X: each column is an x vector"
print X
print
print "Y: each element is a y value"
print y

True formula: y = 2 + x1*5 + x2*4 + noise (if needed)

X: each column is an x vector
[[  1.           1.           1.           1.           1.           1.
    1.           1.           1.           1.        ]
 [ 17.91083053  11.95308736  14.26630328  19.91906186  14.1198285
   14.30245984  14.09699088  18.75599415  15.35029217  18.17591462]
 [  4.67970815   2.46769548   2.43270842   1.37310306   2.64425863
    4.86233748   2.32753645   2.28021862   2.74270834   4.54512193]]

Y: each element is a y value
[ 110.27298524   71.63621874   83.06235008  107.08772155   83.17617701
   92.96164913   81.79510018  104.90084523   89.72229419  111.06006082]


In [ ]:
# Cost function
# Let h_θ(x) be the predicting function: h_θ(x) = w0 + w1*x1 + w2*x2
# where θ = [w0, w1, w2]
# For OLS, cost function: J(θ) = 1/2 sum_over_all_X_y_pairs((h_θ(xi) - yi)^2)

# We want to choose θ so as to minimize J(θ)

# Algorithm:
# Starts with some “initial guess” for θ
# Repeat:
#     Changes θ to make J(θ) smaller:
#        θj := θj - alpha * gradient_of_J_wrt_θj 
#        where j is the j-th component of θ. (θ0 = w0, θ1 = w1, θ2 = w2)  
#     Reminder:
#     - If the gradient is -ve, we're going downslope, we want to move θ forward. 
#       'Minus a negative number' means add, and thus increasing θ.
# until we converge to a value of θ that minimizes J(θ)
#
# Convergence check usually is: L2(gradient) < A_stopping_criteria
# http://stackoverflow.com/questions/13059564/what-should-be-a-generic-enough-convergence-criteria-of-stochastic-gradient-desc

# If we only have ONE single training example:
#
# Gradient of J wrt θj = d J(θ) / d θj = 
# d 1/2 sum_over_all_X_y_pairs((h_θ(xi) - yi)^2)/ d θj = 
# (h_θ(x) - y)x_j

# Learning rate
alpha = 0.01
w = [0,0,0] # Initialize the weight

# h_θ(x) for all training examples
# Example format of yhats: [  99.49519886  111.03223123  109.21172544  104.2711384    90.53712043]
yhats = np.dot(w, X)
ydiffs = yhats - y # (h_θ(x) - y) for all training examples

# (h_θ(x) - y)x0 all training examples
# This is the adjustment for w0
g0 = ydiffs * X[0,:]
g1 = ydiffs * X[1,:]
g2 = ydiffs * X[2,:]

In [31]:
print "Gradient for w0 (1st row), w1 and w2 (last row) for ALL examples."
print "Each column represent an example.  Five in total."
print g0
print g1
print g2

Gradient for w0 (1st row), w1 and w2 (last row) for ALL examples.
Each column represent an example.  Five in total.
[ -99.49519886 -111.03223123 -109.21172544 -104.2711384   -90.53712043]
[-1540.99507572 -1904.41302337 -1750.40553776 -1610.86728187 -1288.93606021]
[-356.54865175 -305.37736894 -498.77999791 -506.19999069 -229.39988431]


In [33]:
# This will give the same result
g = ydiffs * X
print "Gradient for ALL examples."
print "Each row represent a component in the weights (e.g. w0 = 1st row)"
print "Each column represent an example."
print
print g

Gradient for ALL examples.
Each row represent a component in the weights (e.g. w0 = 1st row)
Each column represent an example.

[[  -99.49519886  -111.03223123  -109.21172544  -104.2711384    -90.53712043]
 [-1540.99507572 -1904.41302337 -1750.40553776 -1610.86728187
  -1288.93606021]
 [ -356.54865175  -305.37736894  -498.77999791  -506.19999069
   -229.39988431]]


In [34]:
# (Again)
# Algorithm:
# Starts with some “initial guess” for θ
# Repeat:
#     Changes θ to make J(θ) smaller:
#        θj := θj - alpha * gradient_of_J_wrt_θj 
#        where j is the j-th component of θ. (θ0 = w0, θ1 = w1, θ2 = w2)  
# until we converge to a value of θ that minimizes J(θ)

# If we only have ONE single training example:
#
# Gradient_of_J_wrt_θj = d J(θ) / d θj = 
# d 1/2 sum_over_all_X_y_pairs((h_θ(xi) - yi)^2)/ d θj = 
# (h_θ(x) - y)x_j

# If we have many training example:
#
# Gradient_of_J_wrt_θj = alpha * sum_over_all_examples((h_θ(x) - y)x_j) / number_of_examples
#
# Note: we have to average it out over the examples, otherwise gradient will increase as sample size increases.
g_sum_over_examples = np.sum(g, axis=1)

In [48]:
print "Each column represents the sum of gradient from all examples.  1st column means w0."
print g_sum_over_examples

Each column represents the sum of gradient from all examples.  1st column means w0.
[-0.3942039   0.79810913 -4.0575618 ]


In [51]:
wdelta = -1 * alpha * g_sum_over_examples / size

In [52]:
print wdelta

[  7.88407806e-05  -1.59621825e-04   8.11512360e-04]


In [ ]:
w = w + wdelta
print w

### Linear Regression using Batch Gradient Descent

In [36]:
# Let's put everything under a loop
size = 1000
X, y = genData(size, noise_mean = 0, noise_sd = 0.5)

# Learning rate
alpha = 0.001
stopping_critera = 0.0001
w = [0,0,0] # Initialize the weight
iterations = 1000

for i in range(iterations):
    yhats = np.dot(w, X)
    ydiffs = yhats - y # (h_θ(x) - y) for all training examples
    g = ydiffs * X
    g_sum_over_examples = np.sum(g, axis=1)
    wdelta = -1 * alpha * g_sum_over_examples / size
    
    # Convergence check
    if np.linalg.norm(wdelta) <= stopping_critera:
        break
        
    #print "Iteration", i, "- wdelta =", wdelta
    w = w + wdelta

print "True formula: y = 2 + x1*5 + x2*4 + noise"
print "After", i+1, " iterations, w = ", w

True formula: y = 2 + x1*5 + x2*4 + noise
After 1000  iterations, w =  [ 0.54728746  5.18380913  3.52768918]


### Linear Regression using Stochastic Gradient Descent

- In Batch Gradient Descent, in each loop we update w by batching up the changes from all the examples.
- In Stochastic Gradient Descent, within each loop, we loop through all examples, and update w for each example

In [39]:
# Let's put everything under a loop
size = 1000
X, y = genData(size, noise_mean = 0, noise_sd = 0.2)

# Learning rate
alpha = 0.005
stopping_critera = 0.0001
w = [0,0,0] # Initialize the weight
iterations = 500

for i in range(iterations):
    for m in range(size):
        x = X[:,m] # a single example
        y_single = y[m]
        
        # (h_θ(x) - y) for one training example
        yhat = np.dot(w, x) 
        ydiff = yhat - y_single
        
        # Gradient for one example        
        g = ydiff * x
        
        wdelta = -1 * alpha * g
    
        # Convergence check
        if np.linalg.norm(wdelta) <= stopping_critera:
            break
        
        w = w + wdelta

print "True formula: y = 2 + x1*5 + x2*4 + noise"
print "After", i+1, " iterations, w = ", w

True formula: y = 2 + x1*5 + x2*4 + noise
After 500  iterations, w =  [ 2.05590869  4.9859899   4.00956634]


### Linear Regression using Batch Gradient Descent with Spark!

In [40]:
# Let's put everything under a loop
size = 1000
X, y = genData(size, noise_mean = 0, noise_sd = 0.5)

# Learning rate
alpha = 0.005
stopping_critera = 0.0001
w = [0,0,0] # Initialize the weight
iterations = 500

# Convert X, y into an array of Point object.  Note that each column in X is an x, so we
# need to do the transpose (X.T), otherwise we'll be getting each row of X instead of each column.
points = [Point(one_x, one_y) for one_x, one_y in zip(X.T, y)]
data = sc.parallelize(points).cache()

for i in range(iterations):
    # If we don't use broadcast, the driver has to send out w for each task.
    wbroadcast = sc.broadcast(w)

    # Run this for each single example to get the partial gradient
    # (h_θ(x) - y)x_j
    partial_gradients = data.map(lambda point: (np.dot(wbroadcast.value, point.x)- point.y)*point.x)
    
    # Add up the partial gradient from all the example, and average them out.
    g = partial_gradients.reduce(lambda a, b: a + b)/size
    
    wdelta = -1 * alpha * g
    
    # Convergence check
    if np.linalg.norm(wdelta) <= stopping_critera:
        break
        
    #print "Iteration", i, "- wdelta =", wdelta
    w = w + wdelta

print "True formula: y = 2 + x1*5 + x2*4 + noise"
print "After", i+1, " iterations, w = ", w

True formula: y = 2 + x1*5 + x2*4 + noise
After 500  iterations, w =  [ 0.63756532  5.08123991  4.03859815]


## Logistic Regression

### Background
```
- Let's say we have input x and outcome y, where y is a classification, either 0 or 1. 
- We want to have a function h_θ(x) = Pr(y = 1 | x; θ)

- Options for h_θ(x) (remember: h_θ(x) is a probability):
  - h_θ(x) = θ'X (i.e. h_θ(x) = w0 + w1x1 + w2x2 + ...)
    - Problem: the range of RHS is -inf to +inf
    
  - ln(h_θ(x)) = θ'X 
    - Problem: 0 <= h_θ(x) <= 1; so -inf < ln(h_θ(x)) < 0. The range of LHS != range of RHS
    
  - ln(h_θ(x) / 1 - h_θ(x)) = θ'X
    - Good.  If h_θ(x) = 0, LHS = -inf.  If h_θ(x) = 1, LHS = +inf.
    - So the range of LHS = range of RHS

 -  Take exponent of both sides:
   - exp(ln(h_θ(x) / 1 - h_θ(x))) = exp(θ'X) ==> 
   - h_θ(x) = 1 / (1 + exp(-θ'X))
   
 - Prediction:
   - y_hat = 1 if h_θ(x) >= 0.5
   - y_hat = 0 if h_θ(x) < 0.5
   
 - To solve for θ, we use Gradient Descent.
   - To use GD, we need a cost function.  
   - Cost = high if we make a mistake: y_hat = 0 if actual y = 1; or y_hat = 0 if actual y = 1
   - J(θ) = -y ln(h_θ(x)) - (1-y)lb(h_θ(x))
     - E.g. if y = 1, but h_θ(x) = 0.1 (predict y = 0), Cost = -1 * ln(0.1) = -1 * -2.3 = 2.3
     -      if y = 1, but h_θ(x) = 0.9 (predict y = 1), Cost = -1 * ln(0.9) = -1 * -0.1 = 0.1
 
   - Gradient of J(θ) at each j-th compoent = sum_over_examples[(h_θ(xi) - yi)x_ij]
  
```

In [119]:
def genData(size, noise_mean = None, noise_sd = None):
    x1 = np.random.uniform(10, 20, size)
    x2 = np.random.uniform(1,5, size)

    # True formula: y = 2 + x1*5 + x2*4 + noise
    true_weights = [2, 5, 4]

    # Add the "x_0 = 1" (called the intercept term) so that we can use matrix multiplication
    X = np.vstack((np.ones(size), x1, x2))

    y = np.dot(true_weights, X)

    if noise_mean is not None and noise_sd is not None:
        noise = np.random.normal(noise_mean, noise_sd, size)
        y = y + noise
            
    return X, (y > 85)

In [120]:
size = 500
X, y = genData(size)
train_len = int(size * 0.8)

X_train = X.T[:train_len,:]
y_train = y.T[:train_len].astype(int)

X_test = X.T[train_len:,:]
y_test = y.T[train_len:].astype(int)


In [124]:
# Learning rate
alpha = 0.01
stopping_critera = 0.0001
w = [0,0,0] # Initialize the weight
iterations = 50000

def h(w, x):
    return (1 / (1 + np.exp(-1 * np.dot(w,x.T))))
            
for i in range(iterations):
    #  - Gradient of J(θ) at each j-th compoent = sum_over_examples[(h_θ(xi) - yi)x_ij]
    #    - h_θ(x) = 1 / (1 + exp(-θ'X))
    g = np.dot(h(w, X_train)-y_train.reshape((1,-1)), X_train).flatten()
    wdelta = -1 * alpha * g / size
    
    # Convergence check
    if np.linalg.norm(wdelta) <= stopping_critera:
        break
        
    #print "Iteration", i, "- wdelta =", wdelta
    w = w + wdelta

print "After", i+1, " iterations, w = ", w



After 50000  iterations, w =  [-11.87490136   0.78171116   0.24834292]


In [125]:
y_hat = (h(w, X_test) >= 0.5).astype(int)
print "Precision from sklearn LogisticRegression =", np.average(y_test == y_hat)

Precision from sklearn LogisticRegression = 0.91


In [123]:
from sklearn.linear_model import LogisticRegression

mod = LogisticRegression()
mod = mod.fit(X_train, y_train)
y_predicted = mod.predict(X_test)
print "Precision from sklearn LogisticRegression =", np.average(y_test == y_predicted)

Precision from sklearn LogisticRegression = 0.91
